In [11]:
doc = """
This jupyter notebook is authored by ygg_anderson for the Token Engineering Commons. See appropriate licensing. 🐧 🐧 🐧
"""

import param
import panel as pn
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np
from scipy.stats.mstats import gmean
import os
pn.extension()


APP_PATH = './'

sheets = [
    "Total Impact Hours so far", 
    "IH Predictions", 
    "#8 Jan 1",
    "#7 Dec 18",
    "#6 Dec 4",
    "#5 Nov 20",
    "#4 Nov 6",
    "#3 Oct 23",
    "#2 Oct 9",
    "#1 Sept 24",
    "#0 Sept 7 (historic)",
] + [f"#{i} IH Results" for i in range(9)]
sheets = {i:sheet for i, sheet in enumerate(sheets)}

def read_excel(sheet_name="Total Impact Hours so far", header=1, index_col=0, usecols=None) -> pd.DataFrame:
    data = pd.read_excel(
        os.path.join(APP_PATH, "data", "TEC Praise Quantification.xlsx"),
        sheet_name=sheet_name,
        engine='openpyxl',
        header=header,
        index_col=index_col,
        usecols=usecols
    ).reset_index().dropna(how='any')
    return data

## Tests
impact_hour_data_1 = read_excel()
impact_hour_data_2 = read_excel(sheet_name="IH Predictions", header=0, index_col=0, usecols='A:I').drop(index=19)
pn.Row(impact_hour_data_1.hvplot.table(), impact_hour_data_2.hvplot.table())

Row
    [0] HoloViews(Table)
    [1] HoloViews(Table)

In [12]:
# Load CSTK data
cstk_data = pd.read_csv('CSTK_DATA.csv', header=None).reset_index().head(100)
cstk_data.columns = ['CSTK Token Holders', 'CSTK Tokens']
cstk_data['CSTK Tokens Capped'] = cstk_data['CSTK Tokens'].apply(lambda x: min(x, cstk_data['CSTK Tokens'].sum()/10))
cstk_data

,CSTK Token Holders,CSTK Tokens,CSTK Tokens Capped
0,0,176256,83367.7
1,1,112192,83367.7
2,2,46263,46263.0
3,3,44539,44539.0
4,4,30783,30783.0
...,...,...,...
95,95,570,570.0
96,96,565,565.0
97,97,563,563.0
98,98,560,560.0


In [13]:
class ImpactHoursData(param.Parameterized):
    historic = pd.read_csv('data/IHPredictions.csv').query('Model=="Historic"')
    optimistic =  pd.read_csv('data/IHPredictions.csv').query('Model=="Optimistic"')
    predicted_hours = param.Number(0.5, bounds=(-.5,1.5), step=0.05)
    total_impact_hours = param.Integer(step=100)
    
    def __init__(self, **params):
        super(ImpactHoursData, self).__init__(**params)
        historic = self.historic.set_index('Round')
        optimistic = self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"].set_index('Round')
        predicted = optimistic.copy()
        predicted['Total IH'] = self.predicted_hours * historic[historic["Actual / Predicted"] == "Predicted"]['Total IH'] + (1 - self.predicted_hours) * optimistic['Total IH']
        predicted['Total Hours'] = self.predicted_hours * historic[historic["Actual / Predicted"] == "Predicted"]['Total Hours'] + (1 - self.predicted_hours) * optimistic['Total Hours']
        self.total_impact_hours = int(predicted['Total IH'].max()) 
        
    
    def impact_hours_accumulation(self):
        x = 'End Date'

        historic = self.historic.set_index('Round')
        optimistic = self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"].set_index('Round')
        predicted = optimistic.copy()
        predicted['Total IH'] = self.predicted_hours * historic[historic["Actual / Predicted"] == "Predicted"]['Total IH'] + (1 - self.predicted_hours) * optimistic['Total IH']
        predicted['Total Hours'] = self.predicted_hours * historic[historic["Actual / Predicted"] == "Predicted"]['Total Hours'] + (1 - self.predicted_hours) * optimistic['Total Hours']

        historic_curve = historic.hvplot(x, 'Total IH', rot=45, title='Impact Hours Accumulation Curve 🧼')
        historic_bar = historic.hvplot.bar(x, 'Total Hours', label='Historic')

        optimistic_curve = optimistic.hvplot(x, 'Total IH')
        optimistic_bar = optimistic.hvplot.bar(x, 'Total Hours', label='Optimistic')

        predicted_curve = predicted.hvplot(x, 'Total IH', rot=45, title='Impact Hours Accumulation Curve :)')
        predicted_bar = predicted.hvplot.bar(x, 'Total Hours', label='Predicted')
        
        self.total_impact_hours = int(predicted['Total IH'].max()) 

        return pn.Column(historic_curve * historic_bar * predicted_curve * predicted_bar * optimistic_curve * optimistic_bar, f"<b>Predicted Impact Hours: </b>{round(self.total_impact_hours)}") 

i = ImpactHoursData()

pn.Row(i, i.impact_hours_accumulation)

Row
    [0] Column(margin=5, name='ImpactHoursData', width=300)
        [0] StaticText(value='<b>ImpactHoursData</b>')
        [1] FloatSlider(end=1.5, name='Predicted hours', start=-0.5, step=0.05, value=0.5, value_throttled=0.5)
        [2] IntInput(name='Total impact hours', step=100, value=7406, value_throttled=7406)
    [1] ParamMethod(method)

In [14]:
import numpy as np
class ImpactHoursFormula(param.Parameterized):
    """
    Sem's Formula 🌱 🐝 🍯
    This formala was a collaboration of Sem and Griff for the TEC hatch impact hours formula. 
    https://forum.tecommons.org/t/impact-hour-rewards-deep-dive/90/5
    """
    total_impact_hours = param.Number(round(impact_hour_data_1['Impact Hours'].sum()), constant=True)
    minimum_raise = param.Number(100, bounds=(10, 10000), step=100)
    expected_raise_per_impact_hour = param.Number(25, bounds=(0,200), step=1)
    maximum_impact_hour_rate = param.Number(100, bounds=(0,200), step=1)
    target_raise = param.Number()
    maximum_raise = param.Number()
    
    def __init__(self, **params):
        super(ImpactHoursFormula, self).__init__(**params)
        self.maximum_raise = self.total_impact_hours * self.expected_raise_per_impact_hour * 10
        self.param['maximum_raise'].bounds =  (self.maximum_raise / 10, self.maximum_raise * 10)
        self.param['maximum_raise'].step = self.maximum_raise / 10
        
        self.target_raise = self.maximum_raise / 2
        self.param['target_raise'].bounds =  (self.minimum_raise, self.maximum_raise)
        self.param['target_raise'].step = self.maximum_raise / 10

    def impact_hours_rewards(self):
        expected_raise = self.total_impact_hours * self.expected_raise_per_impact_hour
        if expected_raise > self.maximum_raise:
            expected_raise = self.maximum_raise
        self.param['maximum_raise'].bounds =  (expected_raise, expected_raise * 10)
        self.param['maximum_raise'].step = expected_raise / 10
        if self.target_raise > self.maximum_raise:
            self.target_raise = self.maximum_raise
        self.param['target_raise'].bounds =  (self.minimum_raise, self.maximum_raise)
        self.param['target_raise'].step = self.maximum_raise / 100
        
        x = np.linspace(self.minimum_raise, self.maximum_raise)

        R = self.maximum_impact_hour_rate

        m = self.expected_raise_per_impact_hour
        
        H = self.total_impact_hours

        y = [R* (x / (x + m*H)) for x in x]

        df = pd.DataFrame([x,y]).T
        df.columns = ['Total XDAI Raised','Impact Hour Rate']
        
       
        try:
            expected_impact_hour_rate = df[df['Total XDAI Raised'] > expected_raise].iloc[0]['Impact Hour Rate']
        except:
            expected_impact_hour_rate = df['Impact Hour Rate'].max()
        try:
            target_impact_hour_rate = df[df['Total XDAI Raised'] > self.target_raise].iloc[0]['Impact Hour Rate']
        except:
            target_impact_hour_rate = df['Impact Hour Rate'].max()
        impact_hours_plot = df.hvplot.area(title='Expected and Target Raise', x='Total XDAI Raised',  xformatter='%.0f', hover=True)
        
        return impact_hours_plot * hv.VLine(expected_raise) * hv.HLine(expected_impact_hour_rate) * hv.VLine(self.target_raise) * hv.HLine(target_impact_hour_rate)

    def funding_pools(self):
        x = np.linspace(self.minimum_raise, self.maximum_raise)

        R = self.maximum_impact_hour_rate

        m = self.expected_raise_per_impact_hour
        
        H = self.total_impact_hours

        y = [R* (x / (x + m*H)) for x in x]

        df = pd.DataFrame([x,y]).T
        df.columns = ['Total XDAI Raised','Impact Hour Rate']
        
        # Minimum Results
        minimum_raise = self.minimum_raise
        minimum_rate = df[df['Total XDAI Raised'] > minimum_raise].iloc[0]['Impact Hour Rate']
        minimum_cultural_tribute = self.total_impact_hours * minimum_rate
        
        # Expected Results
        expected_raise = self.total_impact_hours * self.expected_raise_per_impact_hour
        try:
            expected_rate = df[df['Total XDAI Raised'] > expected_raise].iloc[0]['Impact Hour Rate']
        except:
            expected_rate = df['Impact Hour Rate'].max()
            
        expected_cultural_tribute = self.total_impact_hours * expected_rate

        # Target Results
        target_raise = self.target_raise
        try:
            target_rate = df[df['Total XDAI Raised'] > target_raise].iloc[0]['Impact Hour Rate']
        except:
            target_rate = df['Impact Hour Rate'].max()
            
        target_cultural_tribute = self.total_impact_hours * target_rate

        # Funding Pools and Tribute
        funding = pd.DataFrame.from_dict({
            'Mimimum': [minimum_cultural_tribute, minimum_raise-minimum_cultural_tribute],
            'Expected': [expected_cultural_tribute, expected_raise-expected_cultural_tribute],
            'Target': [target_cultural_tribute, target_raise-target_cultural_tribute]}, orient='index', columns=['Culture Tribute', 'Funding Pool'])
        funding_plot = funding.hvplot.bar(title="Expected and Target Funding Pools", stacked=True, ylim=(0,self.maximum_raise),  yformatter='%.0f').opts(color=hv.Cycle(['#0F2EEE', '#0b0a15', '#DEFB48']))

        return funding_plot

In [15]:
impact_hours_rewards = ImpactHoursFormula()

pn.Row(impact_hours_rewards, impact_hours_rewards.impact_hours_rewards, impact_hours_rewards.funding_pools)

Row
    [0] Column(margin=5, name='ImpactHoursFormula', width=300)
        [0] StaticText(value='<b>ImpactHoursFormula</b>...)
        [1] FloatInput(disabled=True, name='Total impact hours', value=2893, value_throttled=2893)
        [2] FloatSlider(end=10000, name='Minimum raise', start=10, step=100, value=100, value_throttled=100)
        [3] FloatSlider(end=200, name='Expected raise p..., step=1, value=25, value_throttled=25)
        [4] FloatSlider(end=200, name='Maximum impact h..., step=1, value=100, value_throttled=100)
        [5] FloatSlider(end=723250, name='Target raise', start=100, step=7232.5, value=361625.0, value_throttled=361625.0)
        [6] FloatSlider(end=723250, name='Maximum raise', start=72325, step=7232.5, value=723250, value_throttled=723250)
    [1] ParamMethod(method)
    [2] ParamMethod(method)